In [ ]:
import os

if not os.path.exists("/content/BraTS2020_TrainingData") or not os.path.exists("/content/BraTS2020_ValidationData"):
  !unzip "/content/drive/MyDrive/Individual Project Final/brats_dataset.zip" > /dev/null

In [ ]:
import numpy as np
from dataset import get_train_test_ids_completed, create_training_gen, create_test_gen
from unet import create_unet
from unet_model import UNetModel

# Intial seed to use for each test where repeatability is required
# This will for example cause train + test data to be split the same way the first time
seed = 42


# Training custom models

This showcases the model's ability to train a u-net model using the data. It uses the BraTS 2020 dataset training data, which is loaded and split internally into a training and test set. The chosen mri types used to train the model should be specified, although it will default to FLAIR and T1ce if not. The loss function can also be specified, and if it uses categorical cross entropy loss, the u-net activation will be changed from sigmoid to softmax, and the class labels will be one-hot encoded. Otherwise, binary masks are created per class, which should be evaluated individually in the loss function. Losses have been implemeted in the MulticlassMetrics class, which also includes other metrics that can be used to evaluate the performance of the model. The default loss used is a combined loss function, which takes the average of the combination of binary cross entropy loss and dice loss, calculated on a per class basis, hence the use of the sigmoid activation.

In [ ]:
# Training a multilabel u-net model using FLAIR, T1ce, and T2 modalities, 
#.. with a combined dice and binary cross entropy loss function 

# Set the intial seed for repeatability
np.random.seed(seed)

model = UNetModel(models_path="/content/models", loss='combined_loss', modalities=['flair', 't1ce', 't2'], slice_interval=50)
model.train_model(0, 1, 1, 2)

Constructing UnetModel.
Using multilabel metrics
147/147 [==============================] - 557s 4s/step - loss: 0.7362 - accuracy: 0.4540 - sensitivity: 0.2048 - specificity: 0.8778 - binary_cross_entropy_per_channel: 0.5030 - dice_loss: 0.9694 - dice_coef_necrotic: 0.0012 - dice_coef_edema: 0.0030 - dice_coef_enhancing: 0.0000e+00 - combined_loss: 0.7362 - val_loss: 0.7066 - val_accuracy: 0.9943 - val_sensitivity: 0.0000e+00 - val_specificity: 1.0000 - val_binary_cross_entropy_per_channel: 0.4383 - val_dice_loss: 0.9750 - val_dice_coef_necrotic: 0.0035 - val_dice_coef_edema: 0.0083 - val_dice_coef_enhancing: 6.3632e-07 - val_combined_loss: 0.7066 - lr: 0.0010


In [ ]:
history = UNetModel(models_path="/content/models",loss='combined_loss', modalities=['flair', 't1ce', 't2'], slice_interval=50)
history.load_model(0)
history.train_model(0, 1, 1,6)
history.evaluate_model()

Constructing UnetModel.
Using multilabel metrics
49/49 [==============================] - 599s 12s/step - loss: 0.8096 - accuracy: 0.3977 - sensitivity: 0.5478 - specificity: 0.7583 - binary_cross_entropy_per_channel: 0.6469 - dice_loss: 0.9722 - dice_coef_necrotic: 0.0039 - dice_coef_edema: 0.0090 - dice_coef_enhancing: 0.0000e+00 - combined_loss: 0.8096 - val_loss: 0.7719 - val_accuracy: 0.9916 - val_sensitivity: 0.1684 - val_specificity: 0.8581 - val_binary_cross_entropy_per_channel: 0.5697 - val_dice_loss: 0.9741 - val_dice_coef_necrotic: 0.0044 - val_dice_coef_edema: 0.0126 - val_dice_coef_enhancing: 0.0000e+00 - val_combined_loss: 0.7719 - lr: 0.0010


37/37 [==============================] - 43s 1s/step - loss: 0.7730 - accuracy: 0.9997 - sensitivity: 0.0597 - specificity: 0.8550 - binary_cross_entropy_per_channel: 0.5669 - dice_loss: 0.9791 - dice_coef_necrotic: 2.8052e-04 - dice_coef_edema: 3.1248e-04 - dice_coef_enhancing: 0.0000e+00 - combined_loss: 0.7730


In [ ]:
# Training a multilabel u-net model using FLAIR and T1ce modalities, 
#.. with a categorical cross entropy loss function
# Labels will be one-hot encoded

model = UNetModel(models_path="/content/models", loss='categorical_crossentropy', modalities=['flair', 't1ce'], slice_interval=50)
model.train_model(0, 1, 2, 6)
model.evaluate_model()

Constructing UnetModel.
Using multilabel metrics
Epoch 1/2
49/49 [==============================] - 532s 11s/step - loss: 0.2387 - accuracy: 0.4545 - sensitivity: 0.0565 - specificity: 0.9704 - binary_cross_entropy_per_channel: 0.2602 - dice_loss: 0.9688 - dice_coef_necrotic: 0.0042 - dice_coef_edema: 0.0090 - dice_coef_enhancing: 0.0031 - combined_loss: 0.6145 - val_loss: 0.1287 - val_accuracy: 0.9908 - val_sensitivity: 3.3047e-04 - val_specificity: 1.0000 - val_binary_cross_entropy_per_channel: 0.0969 - val_dice_loss: 0.9763 - val_dice_coef_necrotic: 1.0197e-05 - val_dice_coef_edema: 0.0087 - val_dice_coef_enhancing: 9.9341e-09 - val_combined_loss: 0.5366 - lr: 0.0010
Epoch 2/2
49/49 [==============================] - 519s 11s/step - loss: 0.1197 - accuracy: 0.4937 - sensitivity: 1.8138e-04 - specificity: 0.9982 - binary_cross_entropy_per_channel: 0.2213 - dice_loss: 0.9721 - dice_coef_necrotic: 1.3150e-04 - dice_coef_edema: 8.0549e-05 - dice_coef_enhancing: 3.5341e-05 - combined_los

37/37 [==============================] - 34s 918ms/step - loss: 0.0246 - accuracy: 0.9998 - sensitivity: 0.0000e+00 - specificity: 1.0000 - binary_cross_entropy_per_channel: 0.0661 - dice_loss: 0.9787 - dice_coef_necrotic: 1.9568e-04 - dice_coef_edema: 1.8666e-04 - dice_coef_enhancing: 0.0000e+00 - combined_loss: 0.5224


In [ ]:
# Training a binary classifier u-net model using FLAIR, T1ce, and T2 modalities, 
#.. with a combined dice and binary cross entropy loss function 

model = UNetModel(models_path="/content/models",
                  loss='combined_loss',
                  binary=True,
                  modalities=['flair'],
                  slice_interval=50)
model.train_model(0, 1, 2, 6)

Constructing UnetModel.
Using binary metrics
Epoch 1/2
49/49 [==============================] - 506s 10s/step - loss: 0.7903 - accuracy: 0.8356 - sensitivity: 0.8836 - specificity: 0.8349 - dice_loss: 0.9524 - combined_loss: 0.7903 - val_loss: 0.7717 - val_accuracy: 0.9339 - val_sensitivity: 0.9753 - val_specificity: 0.9338 - val_dice_loss: 0.9877 - val_combined_loss: 0.7717 - lr: 0.0010
Epoch 2/2
49/49 [==============================] - 494s 10s/step - loss: 0.7138 - accuracy: 0.9788 - sensitivity: 0.3080 - specificity: 0.9807 - dice_loss: 0.9930 - combined_loss: 0.7138 - val_loss: 0.6637 - val_accuracy: 0.9957 - val_sensitivity: 0.1102 - val_specificity: 0.9970 - val_dice_loss: 0.9961 - val_combined_loss: 0.6637 - lr: 0.0010


# Loading pre-trained weights

In [ ]:
# Loading the model with pre-trained weights, found at kaggle.com


model = UNetModel(models_path="/content/models", loss='categorical_crossentropy', modalities=['flair', 't1ce'])
model.load_model(model_path="/content/drive/MyDrive/Individual Project Final/saved_models/pre-trained_ratislav/model_per_class.h5", compile=False)
model.compile_model()
model.evaluate_model()



Constructing UnetModel.
Using multilabel metrics
37/37 [==============================] - 285s 8s/step - loss: 0.0213 - accuracy: 0.9912 - sensitivity: 0.6287 - specificity: 0.9982 - binary_cross_entropy_per_channel: 0.0358 - dice_loss: 0.6572 - dice_coef_necrotic: 0.4067 - dice_coef_edema: 0.5590 - dice_coef_enhancing: 2.0733e-05 - combined_loss: 0.3465


In [ ]:
# Loading binary classifier with FLAIR and T1ce

model = UNetModel(models_path="/content/models", loss='combined_loss', modalities=['flair', 't1ce'], segment_classes={1:'tumor'})
model.load_model(model_path="/content/drive/MyDrive/Individual Project Final/saved_models/binary/flair_t1ce/models/model_job4", compile=False)
model.compile_model()
model.evaluate_model()

Constructing UnetModel.
Using binary metrics
 1/37 [..............................] - ETA: 7:44 - loss: 1.0290 - accuracy: 0.8626 - sensitivity: 0.9752 - specificity: 0.8615 - dice_loss: 0.8817 - combined_loss: 1.0290

KeyboardInterrupt: ignored

# Lime Explanations

In [ ]:
model = UNetModel(models_path="/content/models", loss='categorical_crossentropy', modalities=['flair', 't1ce'], seed=seed)
model.load_model(model_path="/content/drive/MyDrive/Individual Project Final/saved_models/pre-trained_ratislav/model_per_class.h5", compile=False)
model.compile_model()

# !pip install lime
from lime_xai import LimeXAI

lime = LimeXAI(model, seed)

# lime.explain(5, 100, visualize=True)
# lime.explain(7, 1000, visualize=True)
for i in range(20):
  print(i)
  lime.explain(i, 1000, visualize=True, duplicate_channel=0)
  lime.explain(i, 1000, visualize=True, duplicate_channel=1)
  lime.explain(i, 1000, visualize=True, duplicate_channel="mean")
